# Assignment 54 | Pauli matrices: completeness and orthogonality

In [1]:
# define the pauli spin matrices
import numpy as np
import sympy as sp
from sympy import *
from sympy.physics.quantum.dagger import Dagger

def pauli_matrix(s11, s12, s21, s22):
    return Matrix([[s11, s12], [s21, s22]])

args1, args2, args3, args4 = list(zip((1,0,0,1),(0,1,1,0),(0, -sp.I, sp.I, 0),(1,0,0,-1)))

p0, p1, p2, p3 = list(map(pauli_matrix, args1, args2, args3, args4))

In [2]:
p0

Matrix([
[1, 0],
[0, 1]])

In [3]:
p1

Matrix([
[0, 1],
[1, 0]])

In [4]:
p2

Matrix([
[0, -I],
[I,  0]])

In [5]:
p3

Matrix([
[1,  0],
[0, -1]])

In [6]:
# define the basis quaternions
e, i, j, k = [f*g for f, g in list(zip((1, -sp.I, -sp.I, -sp.I), (p0, p1, p2, p3)))]

In [7]:
e

Matrix([
[1, 0],
[0, 1]])

In [8]:
i

Matrix([
[ 0, -I],
[-I,  0]])

In [9]:
j

Matrix([
[0, -1],
[1,  0]])

In [10]:
k

Matrix([
[-I, 0],
[ 0, I]])

### NOTE: In finite dimensions where operators are represented by matrices, the Hermitian adjoint is given by the conjugate transpose (also known as the Hermitian transpose).

Now, let's take a random quaternion, $\mathbf{q}$ according to the below representation: 

$$\mathbf{q} = \sum_{k = 0}^{4}q_{k}\mathbf{\tau}^{k}$$

In [11]:
q_comps = Matrix(np.random.randint(1, 10 + 1) * np.random.random(4))

In [12]:
q_comps

Matrix([
[ 3.40341570069978],
[ 2.88615638021737],
[0.410707299538361],
[  3.7020476720964]])

In [13]:
# to express q in this representation, we multiply its components by the corresponding 
# elements of the representation: e, i, j, k
# we then sum over the resulting matrices to obtain a single matrix representation of q
import functools
q = [q_comp * t for q_comp, t in list(zip(list(q_comps),(e, i, j, k)))]
Q = functools.reduce(lambda m1, m2: m1 + m2, q)
Q

Matrix([
[  3.40341570069978 - 3.7020476720964*I, -0.410707299538361 - 2.88615638021737*I],
[0.410707299538361 - 2.88615638021737*I,    3.40341570069978 + 3.7020476720964*I]])

In [14]:
# define a trace function that automatically simplifies the resulting sympy expression
def trace(M):
    return simplify(Trace(M))

In [15]:
# define equation 13.59 as a function 
def equation_13_59(t, q):
    return Rational(1, 2) * trace(Dagger(t) * q)

In [16]:
# apply equation 13.59 to recover the componenets of q
Matrix(list(map(equation_13_59, (e,i,j,k), (Q, Q, Q, Q))))

Matrix([
[ 3.40341570069978],
[ 2.88615638021737],
[0.410707299538361],
[  3.7020476720964]])

The result above shows exactly what we're looking for, as the resulting components are excatly those we started out with. 

### Now we turn our attention to the orthogonality relation:

$$\frac{1}{2}\text{Tr}(\mathbf{\tau}^{(i)\dagger}\mathbf{\tau}^{(j)}) = \delta_{ij}$$

An efficient way of demonstrating the above is to arrage the basis matrices into a 4 by 1 vector:

$$\mathbf{T} = [\mathbf{\tau}^{(0)}, \mathbf{\tau}^{(1)}, \mathbf{\tau}^{(2)}, \mathbf{\tau}^{(3)}]^{T} $$

We can then express the above orthogonality relation in terms of the outer product:

$$\mathbf{T} \otimes \mathbf{T}$$

That is, orthogonality is respected if - and only if - the outer product of $\mathbf{T}$ with itself yields a 8 by 8 matrix which has entries yielding non-zero traces only along its diagonals with each of these entries being a 4 by 4 matrix of trace 2. 

In [17]:
# define T
T = Transpose(Matrix([[e], [i], [j], [k]]))

In [18]:
T

Matrix([
[ 1,  0],
[ 0,  1],
[ 0, -I],
[-I,  0],
[ 0, -1],
[ 1,  0],
[-I,  0],
[ 0,  I]]).T

In [19]:
# take the outer product of T with T
otr_prod = Dagger(T) * T
otr_prod

Matrix([
[1,  0,  0, -I,  0, 1, -I,  0],
[0,  1, -I,  0, -1, 0,  0,  I],
[0,  I,  1,  0, -I, 0,  0, -1],
[I,  0,  0,  1,  0, I,  1,  0],
[0, -1,  I,  0,  1, 0,  0, -I],
[1,  0,  0, -I,  0, 1, -I,  0],
[I,  0,  0,  1,  0, I,  1,  0],
[0, -I, -1,  0,  I, 0,  0,  1]])

In [20]:
# demonstrate the orthogonality relation holds
# by looping over the outer product
# and applying 1 / 2 trace(M) to each 4 by 4 block matrix
traces = []
for r in range(0, 8, 2):
    for c in range(0, 8, 2):
        tr = Rational(1,2)*trace(otr_prod[r:r+2, c:c+2])
        traces.append(tr)
print(traces)

# put the traces into an array in their natural order
array = np.array(traces)
array.shape = (4, 4)

Traces = Matrix(array)
Traces

[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]


Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])

The above suffices to prove the orthogonality relation holds. 

### Finally, we examine the completeness relation:


In [45]:
# Here is the key observation:
Rational(1, 4) *( (e * Dagger(e)) + (i * Dagger(i)) + (j * Dagger(j)) + (k * Dagger(k)) )

Matrix([
[1, 0],
[0, 1]])

What the above shows us is that, in combintaion with the way we have defined an inner product between 'vectors' on the space of pauli quaternions, we can express *any* quaternion as a linear combination of the $\mathbf{\tau}^{(k)}$. 

To wit, we have

$$\langle{\mathbf{p}}|{ \mathbf{q}}\rangle  = \frac{1}{2}\text{Tr}(\mathbf{p}\mathbf{q})$$

where $\mathbf{p}\mathbf{q}$ is the ordinary product of 2 by 2 matrices. 

From the computation above, we have it that 

$$2\hat{\mathbf{e}} = \frac{1}{2}\sum_{k = 0}^{3}\mathbf{\tau}^{(k)}\mathbf{\tau}^{(k)\dagger}$$

or

$$\hat{\mathbf{e}} = \frac{1}{4}\sum_{k = 0}^{3}\mathbf{\tau}^{(k)}\mathbf{\tau}^{(k)\dagger}$$



Thus, 

$$\mathbf{q} = \hat{\mathbf{e}} \cdot \mathbf{q} = \frac{1}{4}\sum_{k = 0}^{3}\mathbf{\tau}^{(k)}\mathbf{\tau}^{(k)\dagger}\mathbf{q}$$

Now, we know that the $i^{th}$ compenent of $\mathbf{q}$ is given by: 

$$q_{i} = \langle \mathbf{\tau}^{(i)} | \mathbf{q}\rangle = \frac{1}{2}\text{Tr}(\mathbf{\tau}^{(i)\dagger}\mathbf{q})$$

Thus, 

$$q_{i} = \langle \mathbf{\tau}^{(i)} | \mathbf{q}\rangle = \frac{1}{2}\text{Tr}(\mathbf{\tau}^{(i)\dagger}\frac{1}{4}\sum_{k = 0}^{3}\mathbf{\tau}^{(k)}\mathbf{\tau}^{(k)\dagger}\mathbf{q})$$

$$= \frac{1}{8}\text{Tr}(\mathbf{\tau}^{(i)\dagger}\sum_{k = 0}^{3}\mathbf{\tau}^{(k)}\mathbf{\tau}^{(k)\dagger}\mathbf{q})$$

$$= \frac{1}{8}\sum_{k = 0}^{3}\text{Tr}(\mathbf{\tau}^{(i)\dagger}\mathbf{\tau}^{(k)}\mathbf{\tau}^{(k)\dagger}\mathbf{q}))$$

$$= \frac{1}{8}\sum_{k = 0}^{3}\text{Tr}(\mathbf{\tau}^{(i)\dagger}\mathbf{q}))$$

$$=  \frac{1}{8}\sum_{k = 0}^{3}2q_{i}$$

$$= \frac{1}{4}\sum_{k = 0}^{3}q_{i}$$

$$= \frac{4q_{i}}{4}$$

$$= q_{i}$$

In [62]:
pairs = ((e, Dagger(e)), (i, Dagger(i)), (j, Dagger(j)), (k, Dagger(k)))

In [63]:
indices = [(i, j, k, l) for i in range(2) for j in range(2) for k in range(2) for l in range(2)]

In [93]:
tupl_sums = {}
for tupl in indices:
    a = pairs[0][0][tupl[0],tupl[1]]
    b = pairs[0][1][tupl[2],tupl[3]]

    c = pairs[1][0][tupl[0],tupl[1]]
    d = pairs[1][1][tupl[2],tupl[3]]

    e = pairs[2][0][tupl[0],tupl[1]]
    f = pairs[2][1][tupl[2],tupl[3]]

    g = pairs[3][0][tupl[0],tupl[1]]
    h = pairs[3][1][tupl[2],tupl[3]]
        
    tupl_sums[tupl] = (0.5*((a*b) + (c*d) + (e*f) + (g*h)),
                       "a1:{} = a4:{}?".format(tupl[0], tupl[3]),
                       tupl[0] == tupl[3],
                       "a2:{} = a3:{}?".format(tupl[1], tupl[2]),
                       tupl[1] == tupl[2])
                 
    

In [94]:
tupl_sums

{(0, 0, 0, 0): (1.00000000000000, 'a1:0 = a4:0?', True, 'a2:0 = a3:0?', True),
 (0, 0, 0, 1): (0, 'a1:0 = a4:1?', False, 'a2:0 = a3:0?', True),
 (0, 0, 1, 0): (0, 'a1:0 = a4:0?', True, 'a2:0 = a3:1?', False),
 (0, 0, 1, 1): (0, 'a1:0 = a4:1?', False, 'a2:0 = a3:1?', False),
 (0, 1, 0, 0): (0, 'a1:0 = a4:0?', True, 'a2:1 = a3:0?', False),
 (0, 1, 0, 1): (0, 'a1:0 = a4:1?', False, 'a2:1 = a3:0?', False),
 (0, 1, 1, 0): (1.00000000000000, 'a1:0 = a4:0?', True, 'a2:1 = a3:1?', True),
 (0, 1, 1, 1): (0, 'a1:0 = a4:1?', False, 'a2:1 = a3:1?', True),
 (1, 0, 0, 0): (0, 'a1:1 = a4:0?', False, 'a2:0 = a3:0?', True),
 (1, 0, 0, 1): (1.00000000000000, 'a1:1 = a4:1?', True, 'a2:0 = a3:0?', True),
 (1, 0, 1, 0): (0, 'a1:1 = a4:0?', False, 'a2:0 = a3:1?', False),
 (1, 0, 1, 1): (0, 'a1:1 = a4:1?', True, 'a2:0 = a3:1?', False),
 (1, 1, 0, 0): (0, 'a1:1 = a4:0?', False, 'a2:1 = a3:0?', False),
 (1, 1, 0, 1): (0, 'a1:1 = a4:1?', True, 'a2:1 = a3:0?', False),
 (1, 1, 1, 0): (0, 'a1:1 = a4:0?', False, 'a

In [95]:
list(filter(lambda x: x[2] and x[4], tupl_sums.values()))

[(1.00000000000000, 'a1:0 = a4:0?', True, 'a2:0 = a3:0?', True),
 (1.00000000000000, 'a1:0 = a4:0?', True, 'a2:1 = a3:1?', True),
 (1.00000000000000, 'a1:1 = a4:1?', True, 'a2:0 = a3:0?', True),
 (1.00000000000000, 'a1:1 = a4:1?', True, 'a2:1 = a3:1?', True)]